In [2]:
%run ../../Setup.ipynb
import ast
import json
import pandas as pd
import numpy as np
import random
from bdcc.database.connection import database_connector as connector
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

In [3]:
http_status = 200
db_con = connector.DatabaseConnector()
db_con.connect()

matches = list(db_con.get())
db_con.disconnect()

In [4]:
matches

[{'_id': ObjectId('60d33ba83420b8edaaa9cba8'),
  'match_id': 6049289816,
  'radiant_win': True,
  'players': [{'pings': 2,
    'assists': 3,
    'deaths': 1,
    'kills': 21,
    'win': True},
   {'pings': 6, 'assists': 10, 'deaths': 1, 'kills': 0, 'win': True},
   {'pings': 11, 'assists': 7, 'deaths': 4, 'kills': 2, 'win': True},
   {'pings': 0, 'assists': 6, 'deaths': 0, 'kills': 9, 'win': True},
   {'pings': 2, 'assists': 2, 'deaths': 2, 'kills': 1, 'win': True},
   {'pings': 4, 'assists': 1, 'deaths': 14, 'kills': 1, 'win': False},
   {'pings': 12, 'assists': 1, 'deaths': 11, 'kills': 1, 'win': False},
   {'pings': 9, 'assists': 1, 'deaths': 3, 'kills': 0, 'win': False},
   {'pings': 1, 'assists': 1, 'deaths': 2, 'kills': 2, 'win': False},
   {'pings': 1, 'assists': 3, 'deaths': 5, 'kills': 1, 'win': False}]},
 {'_id': ObjectId('60d33ba83420b8edaaa9cbaa'),
  'match_id': 6049281711,
  'radiant_win': True,
  'players': [{'pings': 16,
    'assists': 9,
    'deaths': 4,
    'kills': 1,

In [5]:
feature_list = []

for j in range(len(matches)):
    ping_list_win = []
    ping_list_loss = []
    for i in range(len(matches[j]['players'])):
        if matches[j]['players'][i]['win'] == True:
            ping_list_win.append(matches[j]['players'][i]['pings'])
            
        else:
            ping_list_loss.append(matches[j]['players'][i]['pings'])
            
    median_w = np.mean(ping_list_win)
    mean_w = np.median(ping_list_win)
    std_w = np.std(ping_list_win)
    skew_w = (mean_w - median_w)/std_w
    range_w = np.ptp(ping_list_win) 
    
    median_l = np.mean(ping_list_loss)
    mean_l = np.median(ping_list_loss)
    std_l = np.std(ping_list_loss)
    skew_l = (mean_w - median_w)/std_w
    range_l = np.ptp(ping_list_loss)  
    
    #feature_list.append([1, mean_w, median_w, std_w, skew_w, range_w]) 
    #feature_list.append([0, mean_l, median_l, std_l, skew_l, range_l])  
    r = random.randint(0, 1)
    if(r == 0):
        feature_list.append([1, mean_w, median_w, std_w, skew_w, range_w, mean_l, median_l, std_l, skew_l, range_l])
    else:
        feature_list.append([0, mean_l, median_l, std_l, skew_l, range_l, mean_w, median_w, std_w, skew_w, range_w])
    
dataframe_ping = pd.DataFrame(feature_list, columns =['win', 'mean1', 'median1', 'std1', 'skew1', 'range1', 'mean2', 'median2', 'std2', 'skew2', 'range2'])
dataframe_ping = dataframe_ping.dropna()


In [6]:
y = dataframe_ping["win"]
X = dataframe_ping.drop(["win"], axis = 1)

X_train, X_test, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state = 42)

In [7]:
clf1_p = MLPClassifier(random_state=10, max_iter=10000).fit(X_train, ytrain) #Erhöhung der max_iter um konvergenz zu erreichen
#print("Vorhersage MLPClassifier: ", clf1.predict(x_test))
print("Gesamtgenauigkeit MLPClassifier: ", clf1_p.score(X_test, ytest))
print("Proba: ", clf1_p.predict_proba(X_test))

clf2_p = RandomForestClassifier(random_state=10).fit(X_train, ytrain)
#print("Vorhersage RandomForestClassifier: ", clf2.predict(x_test))
print("Gesamtgenauigkeit RandomForestClassifier: ", clf2_p.score(X_test, ytest))

clf3_p = svm.SVC(random_state=10).fit(X_train, ytrain)
#print("Vorhersage SVM: ", clf3.predict(x_test))
print("Gesamtgenauigkeit SVM: ", clf3_p.score(X_test, ytest))

clf4_p = SGDClassifier(random_state=10).fit(X_train, ytrain)
#print("Vorhersage SGDClassifier: ", clf4.predict(x_test))
print("Gesamtgenauigkeit SGDClassifier: ", clf4_p.score(X_test, ytest))

clf5_p = DecisionTreeClassifier(max_depth = 3).fit(X_train, ytrain) # kein random_state attribut
#print("Vorhersage DecisionTreeClassifier: ", clf5.predict(x_test))
print("Gesamtgenauigkeit DecisionTreeClassifier: ", clf5_p.score(X_test, ytest))

clf6_p = KNeighborsClassifier().fit(X_train, ytrain) # kein random_state attribut
#print("Vorhersage KNeighborsClassifier: ", clf6.predict(x_test))
print("Gesamtgenauigkeit KNeighborsClassifier: ", clf6_p.score(X_test, ytest))

clf7_p = GaussianNB().fit(X_train, ytrain) # kein random_state attribut
#print("Vorhersage GaussianNB: ", clf7.predict(x_test))
print("Gesamtgenauigkeit GaussianNB: ", clf7_p.score(X_test, ytest))

Gesamtgenauigkeit MLPClassifier:  0.66
Proba:  [[5.36021560e-01 4.63978440e-01]
 [9.89189004e-01 1.08109957e-02]
 [7.29440682e-01 2.70559318e-01]
 [5.00987745e-01 4.99012255e-01]
 [2.01063610e-11 1.00000000e+00]
 [9.99793147e-01 2.06853042e-04]
 [9.99411717e-01 5.88283106e-04]
 [9.73385451e-01 2.66145491e-02]
 [9.44175911e-01 5.58240890e-02]
 [1.79703741e-01 8.20296259e-01]
 [1.75960542e-07 9.99999824e-01]
 [2.78533243e-01 7.21466757e-01]
 [9.99986083e-01 1.39170317e-05]
 [8.70060353e-01 1.29939647e-01]
 [1.00000000e+00 1.34416238e-11]
 [1.38539294e-03 9.98614607e-01]
 [9.05906277e-01 9.40937231e-02]
 [7.95969413e-05 9.99920403e-01]
 [9.99157141e-01 8.42858703e-04]
 [9.91103838e-01 8.89616165e-03]
 [8.91246906e-01 1.08753094e-01]
 [5.71575850e-01 4.28424150e-01]
 [2.20534547e-01 7.79465453e-01]
 [9.79437474e-01 2.05625264e-02]
 [4.96935826e-13 1.00000000e+00]
 [9.98512316e-01 1.48768397e-03]
 [9.16545039e-01 8.34549610e-02]
 [9.71827790e-01 2.81722103e-02]
 [9.50712989e-01 4.92870105e-